# Laboration 2 - inlämning del 2 Analys av tweets från bokmässan

## Attribution David Johnsson, Uppsala University

Starta med att ladda in följande moduler och sätt upp visualiseringsmiljön för matplotlib

1. `pandas` 
2. `textmining` 
Funktioner för statistisk textmining, fokuserad på bag-of-words model (som ni inte behöver sätta er in för denna kurs.f För den nyfikne eller vetgirige finns enkla förklaringar exempelvis [här](https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/) eller [här](https://www.geeksforgeeks.org/bag-of-words-bow-model-in-nlp/), en enkel tutorial finns också [här](https://machinelearningmastery.com/gentle-introduction-bag-words-model/)) 
3. `wordcloud` - En visualiseringsmodul för att skapa ordmoln, vilket vi gör i denna laboration.
4. `matplotlib` 
5. `sklearn` -  Scikit-learn,ett pythonbibliotek för maskininlärningsalgoritmer, den kommer vi använda mycket i både laboration 3 och 4.

In [ ]:
pip install nltk 

In [1]:
# Kör denna cell för att ladda in biblioteken och sätta upp vår miljö
import pandas as pd
import nltk as tm
from nltk.corpus import stopwords
import wordcloud
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer
# Sätt upp visualiseringen
%matplotlib inline
matplotlib.pyplot.rcParams['figure.figsize'] = [10, 6]

In [ ]:
tm.download('stopwords')

In [ ]:
stopWords = set(stopwords.words('swedish'))
stopWords

## Analys av Twitterdata från bokmässan

Ni har blivit inhyrda som konsulter för en bokpublicist som vill att du ska ta reda på vilka teman och böcker som har fått mest uppmärksamhet på bokmässan i Göteborg 2016. 

Er uppgift är att via Twitterdata undersöka vilka ämnen som fått speciellt mycket uppmärksamhet för och under bokmässan och presentera ett förslag till företaget du arbetar med vad som är lämpliga debattämnen. 

Fokus här är alltså på att förstå data, vilket är en viktigt del av pre-processering inför mer avacerad dataanalys. 

**F1.** Vad för data är distinkt för twitter och vilken typ av pre-processing tror ni kommer behövas på den typen av data? 

## Data processing

Som alltid behöver vårt data städas, i detta fall är fokus att sortera bort data som antingen inte går att analysera eller inte är intressant från den råtextdata vi fått från Twitter. Den data som givits samlades in från Twitter från maj till september 2016.

Er datafil finns i mappen data i laborationsrepositoriet och heter `twitter_book_fair_data.tsv`.

### Ladda data

En `.tsv` fil betyder att det är en tab-separerad fil med tabelldata (jämfört med ; separerad som vi använt tidigare)

**F2** Starta arbetet med att läsa in filen med read_csv() med följande parametrar:  encoding="utf-8", sep="\t" och spara i en dataframe

**F3** Inspektera den dataframe som skapats med lämpliga funktioner. Ta reda på följande:

Hur ser den ut?
Antal kolumner och rader?
Datatyper?

Glöm inte bort att när du utför operationer på en datafram så sparas ingenting om du inte skapar en variabel som du lagrar dina ändringar i! (alternativt skriver över den dataframe du har genom att sätta parametern inplace = True (default är False).

**F4** Finns det nullvärden i vårt dataset? Varför/varför inte?

**F5.** Hur många tweets i vårt dataset är nämnanden av andra användare (alltså när `@twittername` finns med i tweeten) 

*Hint: Det kan vara till hjälp att använda funktionen `info()`*

**F6.** En kolumn är speciellt intressant för vår **textanalys**, extrahera den från den dataframe vi lagrat all data i och skapa en variabel där du placerar denna data, döp variablen till `tweets_corpus`.

### Emojis

På Twitter är det väldigt vanligt med emojis 👍 ✨ 🐫 🎉 🚀 🤘.

Dessa kan innehålla mycket information som kan vara relevant för vår analys. Dock är det ofta svårt att analysera emojis med hjälp av vanliga verktug för NLP(Natural Language Processig). 

Vi behöver därför ta bort dessa ur vårt utvalda dataset som skapades i uppgiften ovan.

Följande kod utför detta, ni behöver inte bry er om lambda just nu, men vi kommer gå igenom det lite senare i kursen. 

In [ ]:
encode2ascii = lambda x: x.encode('ascii', errors='ignore').decode('utf-8')
clean_tweets = tweets_corpus.apply(encode2ascii)
clean_tweets

**F7.** Hur påverkas kvaliteten på vår analys potentiellt av att ta bort alla emojis? Förklara svaret.

### Ta bort URLs
Det är också vanligt att man på Twitter länkar till olika webbplatser med hjälp av URL:er, när man gör textanalys på twitterdata är det vanligt att delar av dessa URL:er dyker upp som "mest frekventa ord" vilket påverkar vår analys negativs. Dessa behöver därför också tas bort.

In [ ]:
clean_tweets = clean_tweets.str.replace(r'http\S+', '')
clean_tweets

**F8.** Hur kan borttagandet av URL:er pvåerkar analysen och dess kvalitet, förklara svaret.

### Funktion för att hitta mest frekventa ord 

Ett sätt att förstå hur olika metoder för pre-processing påverkar ett dataset kan man räkna de mest förekommande orden efter varje operation som utförs. Eftersom vi kommer vilja utföra denna räkning många gånger under arbetet är de lämpligt att skapa en funktion för det som vi kan anropa flera gånger.

#### Vad är en Term Document Matrix (TDM)?

En TDM är en tabell där antalet unika ord räknas för varje dokument. För att göra detta på vårt Twitterdata är det lämpligt att skapa en TDM där varje tweet är en egen vektor där varje element består av de ord som finns i den tweeten. En tweet med tre unika ord blir alltså en vektor med tre element. 

Nedanstående kod skapar denna TDM i form av en funktion med namn `create_term_document_matrix()`:

**F9** Koden nedan är inte kommenterad, lägg in kommentarer som förklarar vad som sker i koden. (No hittar dokumentationen för CountVectorizer() [här](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) och en kort beskrivning med exempel [här](https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/)

In [ ]:
def create_term_document_matrix(corpus, min_df=1):
    cvec = CountVectorizer(min_df=min_df, stop_words=stopWords)
    tfmatrix = cvec.fit_transform(corpus)
    return pd.DataFrame(data=tfmatrix.toarray(), columns=cvec.get_feature_names())

**F10** Testa vår nya funktion genom att skapa en TDM endast för de tre första raderna i `clean_tweets` som kan sorteras ut med `.head(3)` funktionen. 

In [ ]:
#kod här..
create_term_document_matrix( )

**F11.** Hur många kolumner skapades i TDM:n?

För att hitta de mest frekvent förekommander orden i vår TDM behöver vi räkna ord. Det är också lämpligt med en visualisering över dessa vanligast förekommande ord. Även detta kommer vi behöva göra flera gånger och därför är det återigen lämpligt att definiera en funktion `plot_top_words()` som både räknar och plottar orden i ett stapeldiagram. 

**F12** I nedanstående cell är funktionen definierad, men koden är återigen inte kommenterad, skapa kommentarer (eller skriv i en markdowncell) som förklarar vad funktionen gör. 


In [ ]:
def plot_top_words(tweets, num_word_instances, top_words):
    tdm_df = create_term_document_matrix(tweets, min_df=2)
    word_frequencies = tdm_df[[x for x in tdm_df.columns if len(x) > 1]].sum()
    sorted_words = word_frequencies.sort_values(ascending=False)
    top_sorted_words = sorted_words[:num_word_instances]
    top_sorted_words[:top_words].plot.bar()
    return top_sorted_words

Nu kan vi använda `plot_top_words()` funktionen för att räkna ut de mest förekommande orden i hela vårt corpus, viktigt att ha tålamod dock för det kan ta ett tag. Nedanstående kod utför beräkningen.

In [ ]:
top_words = plot_top_words(clean_tweets, 50, 30)
top_words

**F13** Hur många gånger måste ett ord finnas i corpuset för att finnas med i `top_words` så som den är skriven ovan?

**F14.** Hur många ord plottas i stapeldiagrammet? 

### Små bokstäver

Nästa steg i pre-processingen av vårt dataset (vårt corpus) är att göra om alla bokstäver till små. 

**F15** 

a.Utför ändringen att alla stora bokstäver blir små bokstäver i `clean_tweets` och spara i en ny variabel kallad `tweets_lowered`

b.Varför vill man göra det för vår analys?


**F16** Räkna ut en ny variabel med de mest förekommander (frekventa) orden, döp den till `top_words_lowered`

In [4]:
#Skriv klart denna kodcell för F1.16

top_words_lowered = ...
top_words_lowered

Ellipsis

**F17.** Har något förändrats, vad? Förklara svaret.

För att underlätta att jämföra vad våra ansträngningar får för resultat kan det vara bra att enkelt kunna jämföra olika listor med top_words.

**F18** Skapa en ny dataframe som har två kolumner, en med de 20 mest frekventa orden från`top_words` och en med de 20 mest frekventa orden från `top_word_lowered`. Döp kolumnerna till `Top tweeted clean`och  `Top tweeted lowered`. 

In [ ]:
pd.DataFrame({
    'Top tweeted clean': top_words[0:20].index,
    'Top tweeted lowered': top_words_lowered[0:20].index
})

Ett annnat sätt att göra ungefär samma sak, fast lite mer automatiskt är nedanstående kod som också jämför de första 20 orden. Gör om den så att den istället för att jämföra de 20 mest frekventa orden, jämför de ord som är minst förekommande i de två listorna `top_words`och `top_words_lowered`.

**F19** Vad returnerar nedanstående kodrad om de två listor som jämförs är identiska? Vad returneras om de inte är identiska?

In [ ]:
set(top_words[0:20].index) - set(top_words_lowered[0:20].index)

### Korta ord

Korta ord har ofta inte någon egentlig betydelse, alltså behöver vi inte dessa ord. Typiska sådana ord kan vara ja, jo eller nej. Vi bestämmer oss för att alla ord som är kortare än 3 bokstäver inte innehar någon betydelse i vår analys och tar därmed bort dem. 

**F20** Ta bort alla ord med färre bokstäver än 3(HINT: [regular expressions](https://docs.python.org/3/howto/regex.html)), lägg den nya listan med ord (som inte innehåller ord med färre bokstäver än 3) i en variabel med namn `tweets_low_no_small`



In [ ]:
tweets_low_no_small = ...#din kod här

In [ ]:
#Skapar ny topplista utan korta ord
top_words_low_no_small = plot_top_words(tweets_low_no_small, 50, 30)
top_words_low_no_small

**F21.** Efter att korta ord tagits bort, hur många gånger måste ett ord förekomma i vårt corpus för att hamna i den nya listan enligt ovan? 

### Betydelselösa ord

Stop words är andra ord som inte är korta men som ändå inte har betydelse, dessa kan vara lite besvärligare att identifiera och ta bort. En möjlighet är att helt enkelt skapa en lista med sådana ord och sedan använda den listan för att filtrera ut orden ur ett corpus. Vi har ju redan tagit bort alla ord med färre bokstäver än 3, så sådana behöver vi inte lägga in i listan. 

Nedan är ett exempel på en lista med stoppord som är betydelselösa. 


In [ ]:
my_stop_words = ["och", "det", "att", "i", "en", "jag", "hon", 
                "som", "han", "paa", "den", "med", "var", "sig", 
                "foer", "saa", "till", "aer", "men", "ett", 
                "om", "hade", "de", "av", "icke", "mig", "du", 
                "henne", "daa", "sin", "nu", "har", "inte", 
                "hans", "honom", "skulle", "hennes", "daer", 
                "min", "man", "ej", "vid", "kunde", "naagot", 
                "fraan", "ut", "naer", "efter", "upp", "vi", 
                "dem", "vara", "vad", "oever", "aen", "dig", 
                "kan", "sina", "haer", "ha", "mot", "alla", 
                "under", "naagon", "eller", "allt", "mycket", 
                "sedan", "ju", "denna", "sjaelv", "detta", 
                "aat", "utan", "varit", "hur", "ingen", "mitt", 
                "ni", "bli", "blev", "oss", "din", "dessa", 
                "naagra", "deras", "blir", "mina", "samma", 
                "vilken", "er", "saadan", "vaar", "blivit", 
                "dess", "inom", "mellan", "saadant", "varfoer", 
                "varje", "vilka", "ditt", "vem", "vilket", 
                "sitta", "saadana", "vart", "dina", "vars", 
                "vaart", "vaara", "ert", "era", "vilka"]

När vi skapat vår lista är det dags att skapa en funktion som tar bort dessa från ett dokument. Denna funktion är kodad i cellen nedan. (Igen strunta i lambda för tillfället.)

In [ ]:
remove_stopwords = lambda x: ' '.join(y for y in x.split() if y not in my_stop_words)

Funktionen ovan tar alltså bort stoppord från ett dokument (alltså en tweet), för att ta bort stoppord från hela vårt corpus kan funktionen `.apply()`användas. 

**F22.** Skriv den kod som tar bort alla stoppord från `tweets_low_no_small` och skapar en ny variabel `tweets_low_no_small_stopwords` för corpuset utan stoppord.

In [ ]:
tweets_low_no_small_stopwords = ...#din kod här

In [ ]:
top_words_low_no_small_stopwords = plot_top_words(tweets_low_no_small_stopwords, 50, 30)
top_words_low_no_small_stopwords

**F23.** Efter att stopporden tagits bort, hur många gånger måste ett ord förekomma i vårt corpus för att hamna i den nya listan enligt ovan? 

**F24.** Vad är skillnaderna mellan de frekvent förekommande orden i jämförelse med våra tidigare listor? Skriv den kod som jämför dessa tre listor `top_words_lowered`, `top_words_low_no_small` and `top_words_low_no_small_stopwords`, titta på de första 20 orden i listorna.


### Visualisering och rekommendation

Dags att visualisera vårt resultat och övertyga vår klient om att vi hittat de bästa debattämnena för dem! Här gör vi det genom att skapa ett word cloud där de mest frekventa orden syns bäst. 

Nedanstående kod skapar ett ordmoln för `top_words_low_no_small_stopwords`

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud(max_font_size=40)
wordcloud.fit_words(top_words_no_small_stopwords.to_dict())
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

**F25** Ändra i tidigare kod hur många gånger ett ord minst måste finnas för att det ska inkluderas i ordmolnet. Vad förändras?

**F26** När du tittar på ordmolnet, är det fler ord som borde vara stoppord? Ange några stycken och förklara varför de bör tas bort.

**F27.** Vilket tema rekommenderar ni att publicisten ska ha som debattämne? Förklara svaret. 

**F28.** Ni har nu arbetat med textdata, hur är det annorlunda när det gäller pre-processing jämfört med annan typ av data som är av mer numerisk eller kategorisk karaktär?

---
*När ni besvarat samtliga frågor och all er kod fungerar i enlighet med instruktioner, glöm då inte att lämna in en länk till ert repositorie med den färdiga lösningen körd i era notebooks, senast det datum som är angivet. Infoga också en knapp till Colaboratory om ni använt er av denna miljö*. 

**Glöm inte heller att versionshantera i GitHub så att jag kan följa ert arbete!** 

Lycka till!